In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = ''
os.environ['PINECONE_API_KEY'] = ''
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ["GOOGLE_API_KEY"] = ''

In [15]:
from langchain_community.document_loaders import PyPDFLoader

pages = PyPDFLoader('zinger_burgers.pdf')
list = pages.load()
list

[Document(metadata={'source': 'zinger_burgers.pdf', 'page': 0}, page_content='Popular Types of Zinger Burgers and Their Ingredients\nClassic Zinger Burger\n- Spicy fried chicken fillet (crispy)\n- Lettuce\n- Mayonnaise or spicy mayo\n- Soft burger bun\nCheese Zinger Burger\n- Spicy fried chicken fillet (crispy)\n- Slice of cheese (usually cheddar)\n- Lettuce\n- Mayonnaise or spicy mayo\n- Soft burger bun\nZinger Tower Burger\n- Spicy fried chicken fillet (crispy)\n- Hash brown or onion ring\n- Slice of cheese\n- Lettuce\n- Tomato slice\n- Mayonnaise or spicy mayo\n- Soft burger bun\nBBQ Zinger Burger\n- Spicy fried chicken fillet (crispy)\n- BBQ sauce\n- Lettuce'),
 Document(metadata={'source': 'zinger_burgers.pdf', 'page': 1}, page_content='- Tomato slice\n- Grilled onions (optional)\n- Soft burger bun\nDouble Zinger Burger\n- Two spicy fried chicken fillets (crispy)\n- Double slice of cheese\n- Lettuce\n- Mayonnaise or spicy mayo\n- Soft burger bun\nZinger with Jalapeño\n- Spicy frie

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()

documents = text_splitter.split_documents(list)

documents

[Document(metadata={'source': 'zinger_burgers.pdf', 'page': 0}, page_content='Popular Types of Zinger Burgers and Their Ingredients\nClassic Zinger Burger\n- Spicy fried chicken fillet (crispy)\n- Lettuce\n- Mayonnaise or spicy mayo\n- Soft burger bun\nCheese Zinger Burger\n- Spicy fried chicken fillet (crispy)\n- Slice of cheese (usually cheddar)\n- Lettuce\n- Mayonnaise or spicy mayo\n- Soft burger bun\nZinger Tower Burger\n- Spicy fried chicken fillet (crispy)\n- Hash brown or onion ring\n- Slice of cheese\n- Lettuce\n- Tomato slice\n- Mayonnaise or spicy mayo\n- Soft burger bun\nBBQ Zinger Burger\n- Spicy fried chicken fillet (crispy)\n- BBQ sauce\n- Lettuce'),
 Document(metadata={'source': 'zinger_burgers.pdf', 'page': 1}, page_content='- Tomato slice\n- Grilled onions (optional)\n- Soft burger bun\nDouble Zinger Burger\n- Two spicy fried chicken fillets (crispy)\n- Double slice of cheese\n- Lettuce\n- Mayonnaise or spicy mayo\n- Soft burger bun\nZinger with Jalapeño\n- Spicy frie

In [5]:
from langchain_community.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

c:\Users\Administrator\Desktop\Python\Langchain\GenAi-venv\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
from langchain_pinecone.vectorstores import PineconeVectorStore

index_name = 'pinecone-index'
namespace = 'burger'

#pinecone_db = PineconeVectorStore.from_documents(index_name=index_name, embedding=embeddings, documents=documents, namespace=namespace)
pinecone_db = PineconeVectorStore(index_name=index_name, embedding=embeddings, namespace=namespace)

In [10]:
query = 'Zinger with Avocado'
result = pinecone_db.similarity_search(query, k=1, )
result

[Document(metadata={'page': 2.0, 'source': 'zinger_burgers.pdf'}, page_content='- Soft burger bun\nBacon Zinger Burger\n- Spicy fried chicken fillet (crispy)\n- Crispy bacon strips\n- Slice of cheese\n- Lettuce\n- Tomato slice\n- Mayonnaise or BBQ sauce\n- Soft burger bun\nZinger with Avocado\n- Spicy fried chicken fillet (crispy)\n- Sliced avocado\n- Lettuce\n- Tomato slice\n- Mayonnaise or ranch dressing\n- Soft burger bun')]

In [38]:
from langchain_community.llms import Ollama

llm = Ollama(model='gemma2:2b-instruct-q4_0')

llm

Ollama(model='gemma2:2b-instruct-q4_0')

In [29]:
from langchain_openai import ChatOpenAI
openllm = ChatOpenAI(model='gpt-4o-mini')
openllm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000017747994BB0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000017747996BC0>, root_client=<openai.OpenAI object at 0x0000017747977F40>, root_async_client=<openai.AsyncOpenAI object at 0x0000017747994BE0>, model_name='gpt-4o-mini', openai_api_key=SecretStr('**********'), openai_proxy='')

In [30]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """Answer the following question according to provided context:
    <context>
    {context}
    </context>
    
    Question: {input}"""
)

In [31]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(openllm ,prompt)

In [32]:
retriver = pinecone_db.as_retriever()
retriver

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x0000017740C8CB50>)

In [33]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriver, document_chain)

In [34]:
response = retrieval_chain.invoke({"input": "What are ingredients in BBQ Zinger Burger?"})
response['answer']

'The ingredients in a BBQ Zinger Burger are:\n\n- Spicy fried chicken fillet (crispy)\n- BBQ sauce\n- Lettuce\n- Soft burger bun'